### Introduction

This application is for encoding and decoding a series of uncompressed audio files (WAV files). The application must run on a Jupyter notebook is based on the lossless data compression method ‘Rice coding’.

In [12]:
!pip3 install bitstream
!pip3 install pathlib
!pip3 install soundfile
!pip3 install scipy

In [232]:
import io
import soundfile as sf
import pathlib
import sys
from scipy.io import wavfile

##Path variables
sound1_path = 'Sounds/Sound1.wav'
sound2_path = 'Sounds/Sound2.wav'

def read_file(path):
    ''' Function that uses wavfile library'''
    ''' Returns samples and sampling rate'''
    
    ## read and print file information
    samplerate, data = wavfile.read('./Sounds/Sound1.wav')
    print("Sampling Frequency in Hz=", samplerate, "max(x)=", np.max(data))
    
    #return samplerate and data
    return [samplerate, data]

## Reference to predictor:
[1] Springer Nature Switzerland AG 2020 G. Schuller, Filter Banks and Audio Coding, https://doi.org/10.1007/978-3-030-51249-1. Page 161

In [250]:
import numpy as np

def Predictor_decode(e,L,h):
    ''' Loss Predictor Decoding Function'''
    
    #L: Predictor length
    #h: starting values for the L predictor coefficients
    #e: the prediction error
    
    
    xrek = np.zeros(len(e))
    for n in range(L, len(e)):
        
        xrekvec = xrek[n - L + np.arange(L)]
        
        P = np.dot(np.flipud(xrekvec), h)
        
        P = round(P)
        
        xrek[n] = e[n] + P
        h = h + 1.0* e[n]* np.flipud(xrekvec) / (0.1 + np.dot(xrekvec,xrekvec))

        
    return np.hstack((xrek[L:], np.zeros(L)))

def Predictor_encode(x,L,h):
    '''Computes the NLMS lossless predictor'''
    
    #arguments: x: input signal (mono)
    #L: Predictor lenght
    #h: starting values for the L predictor coefficients
    #returns: e, the prediction error
    
    x=np.hstack((np.zeros(L),x)); #make same starting conditions as decoder 
    e=np.zeros(len(x));
    for n in range(L, len(x)):
        #prediction error and filter, using the vector of reconstructed samples, 
        xrekvec=x[n-L+np.arange(L)]
        P=np.dot(np.flipud(xrekvec), h);
        #quantize and de−quantize by rounding to the nearest integer: 
        P=round(P)
        #prediction error:
        e[n]=x[n]-P
        #NLMS update:
        h = h + 1.0 * e[n] * np.flipud(xrekvec)/(0.1+np.dot(xrekvec,xrekvec))
    return e

In [214]:
import numpy as np
import math


def Ricer_encode(S, k, signed):
    ''' Does Rice Encoding'''
    
    
    M = 2**k;
    n = int(S)
    
    
    if signed and n > 0:
        n = 2*n
    elif signed and n < 0:
        n = 2 * abs(n) - 1
    
    r = encode_golomb(abs(n), M, signed)
    
      
    return r

def min_binary(s):
    '''Removes leading zeros'''
    bits = s.lstrip("0") 
    
    if not bits:
        bits = '0'
        
    return bits

def remainder_truncate(r, k):
    '''Truncates remainder'''
    rem = bin(r)
    return rem.split('b')[1].zfill(k)

def encode_golomb(n, M, signed):
    '''Returns a string value of bits'''

    
    q1 = (n // M)
    q = math.floor(q1)
    
    unary_code = unary(q)
    
    
    k1 = math.log(M, 2)
    k = math.ceil(k1)
    
    r = n % M
    
    
    #truncate remainder
    r1 = remainder_truncate(r, k)
    result = unary_code + r1
        
    return result
        


def unary(q):
    '''Converts int to unary representation'''
    A = []
  
    for i in range(q):
        A.append(1)
      
    A.append(0)
  
    B = [str(k) for k in A]
  
    C = "".join(B)
    
    
    return C





def Ricer_decode(S, k, signed):
    '''Reverses rice coding'''
    '''Accepts an array of bit strings'''
    M = 2**k;
        
    r = decode_rice(S, M, signed)

        
    return r

def decode_rice(S, m, signed):
    ''' decode rice bit string'''
    ''' Returns an decoded integer'''

    
    code = [str(i) for i in S]

    
    k1 = math.log(m, 2)
    k = math.ceil(k1)
    q = 0
    
    for i in range(len(code)-k):
        if int(code[i]) == 1:
            q = q + 1
        else:
            break
            
    for i in range(q):
        code.pop(0)
        
    code1 = [str(i) for i in code]
    code2 = "".join(code1)
    
    n = 0
    r1 = []
    
    for i in range(len(code)):
        r1.append(code[i])
        
    r2 = [str(i) for i in r1]
    r3 = "".join(r2)
    r = int(r3, 2)

    n = q * m + r

    ## adjust for +ve and negative values
    if n % 2 == 1 and signed:
        n = (n+1)/2*(-1)
    elif n%2 == 0 and signed:
        n = n/2


    return int(n)

    

                      

In [215]:
#should give back -23
S = "1101101"
n = Ricer_decode(S,4, signed=True)
print("{} converts to {}".format(S,n))

#should return 0b1101101
val = -23
r = Ricer_encode(val, 4, signed=True)
print("{} converts to {}".format(val,r))

1101101 converts to -23
-23 converts to 1101101


In [220]:
import numpy as np
import bitstream


class rice_tag(object):
    '''Rice codec tag type'''
    def __init__(self, b, signed):
        self.b = b 
        self.signed = signed


def write_factory(instance):
    '''Writes rice signed stream'''
    b = instance.b
    signed = instance.signed
    m = 2**instance.b
    def write_uint(stream, data):
        if isinstance(data, list) or isinstance(data, np.ndarray):

            for i in data:
                write_uint(stream, i)
        else:
            integer = int(data) 
            
            rice = Ricer_encode(integer, b, signed = signed)
            bools = [i=='1' for i in rice] 
            stream.write(bools, bool)
            
                    
    return write_uint

def read_uint_factory(instance):
    '''Reads rice signed stream'''
    signed = instance.signed
    b = instance.b
    m = 2**instance.b
    def read_uint(stream, n=None):
        if n is None:
            end = False
            start = True
            have_quotient = False
            bools = []
            while not end:
                try:
                    val = stream.read(bool)
                    
                    
                    
                    if (val and not have_quotient):
                        bools.append(val)
                        start = False
                    elif (not val and not have_quotient):
                        bools.append(val)
                        have_quotient = True
                        
                    if(have_quotient):
                        bits = b
                        
                        if b == 0:
                            bits = bits + 1
                            
                        for i in range(bits):
                            bools.append(stream.read(bool)) 
                        end = True
                              
                
                except:
                    return None
                
                
  
         
            # initialization of string to ""
            S = "".join([str(int(i)) for i in bools])  
            r = Ricer_decode(S, b, signed =signed)

            return r
 
        else:
            integers = [read_uint(stream) for i in range(n)]
            return integers
    return read_uint

#register to bit stream
bitstream.register(rice_tag, writer=write_factory, reader=read_uint_factory)

In [218]:
from bitstream import BitStream

#test custom bitstream
origs=[3 ,  2  , 0 , -4  , 0  , 1 , -2 ,  1,  -1,  -1 , -1 ,  1  ,-1  , 2 , -1 ,  0  ,-3 ,  2,
  -1 ,-11,  -1,  -7,  -3  , 0 , -5  ]

ricecode=rice_tag(2,signed = True)
riceencoded= BitStream(origs, ricecode)

for index in origs:
    print("Index:␣", index, "Rice␣code:␣", BitStream(index, ricecode))

Index:␣ 3 Rice␣code:␣ 1010
Index:␣ 2 Rice␣code:␣ 1000
Index:␣ 0 Rice␣code:␣ 000
Index:␣ -4 Rice␣code:␣ 1011
Index:␣ 0 Rice␣code:␣ 000
Index:␣ 1 Rice␣code:␣ 010
Index:␣ -2 Rice␣code:␣ 011
Index:␣ 1 Rice␣code:␣ 010
Index:␣ -1 Rice␣code:␣ 001
Index:␣ -1 Rice␣code:␣ 001
Index:␣ -1 Rice␣code:␣ 001
Index:␣ 1 Rice␣code:␣ 010
Index:␣ -1 Rice␣code:␣ 001
Index:␣ 2 Rice␣code:␣ 1000
Index:␣ -1 Rice␣code:␣ 001
Index:␣ 0 Rice␣code:␣ 000
Index:␣ -3 Rice␣code:␣ 1001
Index:␣ 2 Rice␣code:␣ 1000
Index:␣ -1 Rice␣code:␣ 001
Index:␣ -11 Rice␣code:␣ 11111001
Index:␣ -1 Rice␣code:␣ 001
Index:␣ -7 Rice␣code:␣ 111001
Index:␣ -3 Rice␣code:␣ 1001
Index:␣ 0 Rice␣code:␣ 000
Index:␣ -5 Rice␣code:␣ 11001


In [219]:
#should give back 0
S = "1010"
n = Ricer_decode(S,2, signed=True)
print("{} converts to {}".format(S,n))

1010 converts to 3


### COMPRESSOR FUNCTION:

Application to compress the file as specified and pass to the compressor(path, b) function.

### REFERENCE
[1] Springer Nature Switzerland AG 2020 G. Schuller, Filter Banks and Audio Coding, https://doi.org/10.1007/978-3-030-51249-1. Page 161-163

In [279]:
import numpy as np
import os
import pickle
import struct
from bitstream import BitStream


def compressor(path, k, optimize=False):
    '''Compression Function'''
    '''Called while setting the k value of the rice parameter M = 2**k'''
    '''Returns compressed file in same directory as original'''
    
    fs, x = read_file(path)
    name,ext=os.path.splitext(path)
    encfile=name+'_Enc.ex2'

    try:
        channels=x.shape[1] #number of channels, needs to be 2 for stereo (2 columns in x)
    except IndexError:
        channels=1 # 1 for mono, make x also 2−dimensional (chan is last dim): 
        x=np.expand_dims(x,axis=-1)
    
    N=int(fs*20e-3) #fs∗20ms=640, number of samples per rice coder block
    L=10 #Predictor order

    with open(encfile, 'wb') as codedfile: #open compressed file
        pickle.dump(fs, codedfile, protocol=-1) #write sampling rate 
        pickle.dump(channels, codedfile, protocol=-1) #write number of channels

        for chan in range(channels): #loop over channels:
            print("channel ", chan)
            print("NLMS Predictor:")
            
            h=np.zeros(L)
            e=Predictor_encode(x[:,chan],L,h) #compute the NLMS predicton error 
            print("len(errors)", len(e))
            print("mean error: ", np.mean(np.abs(e), axis=0))
         
            
            numblocks=len(e)//N
            prederror=np.reshape(e[:numblocks*N], (N,numblocks), order='F') 
            print("number of blocks=", numblocks)
            
            
            if chan==0: 
                pickle.dump(numblocks, codedfile, protocol=-1) #write number of blocks
                pickle.dump(k, codedfile, protocol=-1) #write the k value
                pickle.dump(optimize, codedfile, protocol=-1) #write the k value

            print("Compressing using Rice Coding with k-value {}:".format(k))
            
            #Rice coefficients         
            meanabs=np.mean(np.abs(prederror),axis=0)    
            ricecoefff=np.clip(np.floor(np.log2(meanabs)),0,None)      
            ricecoeffc=np.clip(np.ceil(np.log2((meanabs+1)*2/3)),0,None) 
            ricecoeff=np.round((ricecoeffc+ricecoefff)/2.0).astype(np.int8) #integer, 8bit 
            print("ricecoeff=", ricecoeff) 
            s=struct.pack('b'*int(len(ricecoeff)),*ricecoeff)
            
            #write coefficients
            pickle.dump(s, codedfile, None)
            
            #prederror = np.concatenate((prederror, np.zeros((4,numblocks))), axis=0)
            
            for block in range(numblocks): #loop across blocks:
                if (block%100==0): 
                    print("block:",block)
                
                #Set K Value
                coeff = k
                
                #Computer k value or user set
                if optimize:
                    coeff = ricecoeff[block]
                
                signedrice = rice_tag(coeff, signed=True)
                errors = prederror[:,block]
                    
                errors = np.concatenate((errors, np.zeros((4,))), axis=0)
                
                stream = BitStream(errors.astype(np.int32), signedrice)       
                prederrors=stream.read(bytes, np.floor(len(stream)/8.0))
                pickle.dump(prederrors, codedfile, None)
                
            
        #close file   
        codedfile.close()        

### Testing the compressor:

All Files can be found in the Sounds directory as Sound*_Enc.ex2. The compression follows the flac compression method with the NLMS predictor.

In [273]:
## compress sound1 with k = 4
compressor(sound1_path, 4)

Sampling Frequency in Hz= 44100 max(x)= 7897
channel  0
NLMS Predictor:
len(errors) 501032
mean error:  29.341371409410975
number of blocks= 568
Compressing using Rice Coding with k-value 4:
ricecoeff= [0 0 2 4 4 4 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 4 3 4 3 3 3 2 2 2 2 2 2 2 2 0 0
 0 0 2 2 2 2 2 2 2 2 4 6 8 4 4 4 4 4 4 6 6 6 6 6 6 5 4 4 4 5 6 6 6 6 6 6 7
 8 8 7 7 6 6 6 5 5 6 7 7 7 7 7 6 6 6 8 7 6 5 6 6 6 6 6 5 5 5 4 4 4 4 3 3 4
 4 6 6 6 6 6 6 6 5 4 4 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 2 2 2 2 0
 2 0 0 0 0 0 0 2 2 3 5 6 6 6 6 6 8 8 8 7 6 6 6 5 4 4 6 6 6 6 6 5 2 2 2 6 7
 6 6 6 5 5 6 6 5 5 4 4 4 3 3 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 2 3 4 6 6 6 5 4 2 2 3 5 6 6 5 4 4 4 5 5 6 5 4 4 3
 4 5 5 6 7 6 6 4 3 3 5 6 5 5 6 6 5 4 3 2 2 2 6 7 5 6 6 6 6 5 4 2 2 6 6 4 4
 3 3 4 5 5 5 4 4 4 4 3 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2

In [274]:
## compress sound1 with k = 4
compressor(sound2_path, 4)

Sampling Frequency in Hz= 44100 max(x)= 7897
channel  0
NLMS Predictor:
len(errors) 501032
mean error:  29.341371409410975
number of blocks= 568
Compressing using Rice Coding with k-value 4:
ricecoeff= [0 0 2 4 4 4 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 4 3 4 3 3 3 2 2 2 2 2 2 2 2 0 0
 0 0 2 2 2 2 2 2 2 2 4 6 8 4 4 4 4 4 4 6 6 6 6 6 6 5 4 4 4 5 6 6 6 6 6 6 7
 8 8 7 7 6 6 6 5 5 6 7 7 7 7 7 6 6 6 8 7 6 5 6 6 6 6 6 5 5 5 4 4 4 4 3 3 4
 4 6 6 6 6 6 6 6 5 4 4 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 2 2 2 2 0
 2 0 0 0 0 0 0 2 2 3 5 6 6 6 6 6 8 8 8 7 6 6 6 5 4 4 6 6 6 6 6 5 2 2 2 6 7
 6 6 6 5 5 6 6 5 5 4 4 4 3 3 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 2 3 4 6 6 6 5 4 2 2 3 5 6 6 5 4 4 4 5 5 6 5 4 4 3
 4 5 5 6 7 6 6 4 3 3 5 6 5 5 6 6 5 4 3 2 2 2 6 7 5 6 6 6 6 5 4 2 2 6 6 4 4
 3 3 4 5 5 5 4 4 4 4 3 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2

In [275]:
## compress sound1 with calculating k values per block
compressor(sound1_path, 4, optimize=True)

Sampling Frequency in Hz= 44100 max(x)= 7897
channel  0
NLMS Predictor:
len(errors) 501032
mean error:  29.341371409410975
number of blocks= 568
Compressing using Rice Coding with k-value 4:
ricecoeff= [0 0 2 4 4 4 3 3 3 4 3 3 3 3 3 3 3 3 3 3 3 4 3 4 3 3 3 2 2 2 2 2 2 2 2 0 0
 0 0 2 2 2 2 2 2 2 2 4 6 8 4 4 4 4 4 4 6 6 6 6 6 6 5 4 4 4 5 6 6 6 6 6 6 7
 8 8 7 7 6 6 6 5 5 6 7 7 7 7 7 6 6 6 8 7 6 5 6 6 6 6 6 5 5 5 4 4 4 4 3 3 4
 4 6 6 6 6 6 6 6 5 4 4 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 2 2 2 2 0
 2 0 0 0 0 0 0 2 2 3 5 6 6 6 6 6 8 8 8 7 6 6 6 5 4 4 6 6 6 6 6 5 2 2 2 6 7
 6 6 6 5 5 6 6 5 5 4 4 4 3 3 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 2 3 4 6 6 6 5 4 2 2 3 5 6 6 5 4 4 4 5 5 6 5 4 4 3
 4 5 5 6 7 6 6 4 3 3 5 6 5 5 6 6 5 4 3 2 2 2 6 7 5 6 6 6 6 5 4 2 2 6 6 4 4
 3 3 4 5 5 5 4 4 4 4 3 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2

### DECOMPRESSOR FUNCTION

### REFERENCE
[1] Springer Nature Switzerland AG 2020 G. Schuller, Filter Banks and Audio Coding, https://doi.org/10.1007/978-3-030-51249-1. Page 161-165

In [276]:
from scipy.io.wavfile import write

##Paths to encoded files
sound1_Enc_path = 'Sounds/Sound1_Enc.ex2'
sound2_Enc_path = 'Sounds/Sound2_Enc.ex2'

def decompressor(path):
    '''Decompresses flac compressed wav file'''
    
    ##Get file name and set decoded name
    name,ext=os.path.splitext(path)
    decfile=name.split('_')[0] +'_Dec.wav'
    
    L=10 #Predictor order

    with open(path, 'rb') as encodedfile: #open compressed file
   
        #load file parameters
        fs=pickle.load(encodedfile)
        channels=pickle.load(encodedfile)
        numblocks=pickle.load(encodedfile)
        k=pickle.load(encodedfile)
        optimize = pickle.load(encodedfile)
        
        N=int(fs*20e-3) #fs∗20ms=640, number of samples per rice coded block 
        
        print("numblocks=", numblocks)
        print("fs=", fs, "channels=", channels, ) 
        print("k_value=", k)
        
        #Holds the data
        xrek=np.zeros((numblocks*N, channels))
        for channel in range(channels): #loop over channels:
            print("channel ", channel)
            
            ricecoeffcomp=pickle.load(encodedfile);
            ricecoeff =struct.unpack( 'B' * len(ricecoeffcomp), ricecoeffcomp)
            print("len(ricecoeff)=", len(ricecoeff))
            print("ricecoeff=", ricecoeff) 
            prederrordec=np.zeros(N*numblocks)
            
            for block in range(numblocks): #loop across blocks:
                if (block%100==0): 
                    print("Block number:",block)
                    
                prederrors=pickle.load(encodedfile) #Rice coded block samples
           
                coeff = k
            
                if optimize:
                    coeff = ricecoeff[block]
                    
                signedrice=rice_tag(coeff,signed=True)
                prederrorrice = BitStream()
                prederrorrice.write(prederrors) 
                
                prederrordec[block*N:(block+1)*N]=prederrorrice.read(signedrice, N)
                
                
            print("NLMS␣prediction:");
            
            h=np.zeros(L)
            prederrordec=prederrordec*1.0 #convert to float to avoid overflow 
            print("len(prederrordec)=", len(prederrordec)) 
            xrek[:len(prederrordec),channel] =Predictor_decode(prederrordec,L,h)
            
    
        
        write(decfile, fs, xrek.astype(np.int16))
                
            
        
            
    encodedfile.close()
                
                    
                
                    
                

### Test the decompressor

In [277]:
## decompress Sound1
decompressor(sound1_Enc_path)

numblocks= 568
fs= 44100 channels= 1
k_value= 4
channel  0
len(ricecoeff)= 568
ricecoeff= (0, 0, 2, 4, 4, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 4, 6, 8, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 5, 4, 4, 4, 5, 6, 6, 6, 6, 6, 6, 7, 8, 8, 7, 7, 6, 6, 6, 5, 5, 6, 7, 7, 7, 7, 7, 6, 6, 6, 8, 7, 6, 5, 6, 6, 6, 6, 6, 5, 5, 5, 4, 4, 4, 4, 3, 3, 4, 4, 6, 6, 6, 6, 6, 6, 6, 5, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 3, 5, 6, 6, 6, 6, 6, 8, 8, 8, 7, 6, 6, 6, 5, 4, 4, 6, 6, 6, 6, 6, 5, 2, 2, 2, 6, 7, 6, 6, 6, 5, 5, 6, 6, 5, 5, 4, 4, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 6, 6, 6, 5, 4, 2, 2, 3, 5, 6, 6, 5, 4, 4, 4, 5, 5, 6, 5, 4, 4, 3, 4, 5, 5, 6, 7, 6, 6, 

In [278]:
## decompress Sound1
decompressor(sound2_Enc_path)

numblocks= 568
fs= 44100 channels= 1
k_value= 4
channel  0
len(ricecoeff)= 568
ricecoeff= (0, 0, 2, 4, 4, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 4, 6, 8, 4, 4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 5, 4, 4, 4, 5, 6, 6, 6, 6, 6, 6, 7, 8, 8, 7, 7, 6, 6, 6, 5, 5, 6, 7, 7, 7, 7, 7, 6, 6, 6, 8, 7, 6, 5, 6, 6, 6, 6, 6, 5, 5, 5, 4, 4, 4, 4, 3, 3, 4, 4, 6, 6, 6, 6, 6, 6, 6, 5, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 3, 5, 6, 6, 6, 6, 6, 8, 8, 8, 7, 6, 6, 6, 5, 4, 4, 6, 6, 6, 6, 6, 5, 2, 2, 2, 6, 7, 6, 6, 6, 5, 5, 6, 6, 5, 5, 4, 4, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3, 4, 6, 6, 6, 5, 4, 2, 2, 3, 5, 6, 6, 5, 4, 4, 4, 5, 5, 6, 5, 4, 4, 3, 4, 5, 5, 6, 7, 6, 6, 